In [1]:
import gzip
import pandas as pd
import goslate
import json
import requests

import nltk
import string
from nltk.stem.porter import *
from nltk.corpus import stopwords
nltk.download("stopwords")
from textblob import TextBlob

from sklearn import linear_model
from collections import defaultdict
from datetime import datetime
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Note**: Use cd .. or pwd commands to read the data files if required.

In [2]:
cd ..

D:\OneDrive - Trường ĐH CNTT - University of Information Technology\HKV\Công nghệ dữ liệu lớn\IE212.O11.Group7


In [3]:
data_places = []
with gzip.open("data/places_TX.json.gz") as f:
    for l in f:
        data_places.append(eval(l))

In [4]:
len(data_places), data_places[:1]

(115164,
 [{'name': 'Polo Club Cranbrook',
   'price': None,
   'address': ['14531 Ella Blvd', 'Houston, TX 77014'],
   'hours': [['Monday', [['9:00 am--6:00 pm']]],
    ['Tuesday', [['9:00 am--6:00 pm']]],
    ['Wednesday', [['9:00 am--6:00 pm']], 1],
    ['Thursday', [['9:00 am--6:00 pm']]],
    ['Friday', [['9:00 am--6:00 pm']]],
    ['Saturday', [['10:00 am--5:00 pm']]],
    ['Sunday', [['1:00--5:00 pm']]]],
   'phone': '(281) 872-1222',
   'closed': False,
   'gPlusPlaceId': '111230142112807464303',
   'gps': [29.979443, -95.439439]}])

In [5]:
df_data_places = pd.DataFrame.from_dict(data_places)

In [6]:
df_data_places

,name,price,address,hours,phone,closed,gPlusPlaceId,gps
0,Polo Club Cranbrook,None,"[14531 Ella Blvd, Houston, TX 77014]","[[Monday, [['9:00 am--6:00 pm']]], [Tuesday, [...",(281) 872-1222,False,111230142112807464303,"[29.979443, -95.439439]"
1,La Quinta Inn & Suites Austin Round Rock North,None,"[2004 N Interstate Highway 35, Round Rock, TX ...",None,(512) 255-6666,False,111646207049589408629,"[30.52977, -97.691897]"
2,Coco Beach,None,"[12159 Valliant St, San Antonio, TX 78216]","[[Monday, [['2:00 pm--2:00 am']]], [Tuesday, [...",(210) 341-5330,False,100112847698256295619,"[29.552529, -98.49745]"
3,RadioShack,$$,"[4522 Fredericksburg Rd, San Antonio, TX 78201]","[[Monday, [['10:00 am--9:00 pm']]], [Tuesday, ...",(800) 843-7422,False,106768567721430142078,"[29.492631, -98.552204]"
4,Hampton Inn & Suites Port Arthur,None,"[7660 Memorial Blvd, Port Arthur, TX 77642]",None,(409) 722-6998,False,116191643129206359222,"[29.939667, -93.98328]"
...,...,...,...,...,...,...,...,...
115159,WINX Lash and Makeup Studio,None,"[1973 W Gray St #23, Houston, TX 77019]","[[Monday, [['Closed']]], [Tuesday, [['10:00 am...",(832) 275-9667,False,108538528430034252187,"[29.752956, -95.406743]"
115160,Eastside Tattoo And Piercing Studio,None,"[4819 Columbia Ave, Dallas, TX 75226]","[[Monday, [['1:00--10:00 pm']]], [Tuesday, [['...",(214) 828-9555,False,110544559460583713987,"[32.794887, -96.76422]"
115161,Fritz's Restaurant,None,"[105 Ruthlynn Dr, Longview, TX 75605]","[[Monday, [['11:00 am--2:00 pm']]], [Tuesday, ...",(903) 553-1161,False,110967995566557704067,"[32.530141, -94.743552]"
115162,The Yard Depot,None,"[21202 Northwest Fwy, Cypress, TX 77429]","[[Monday, [['7:00 am--6:00 pm']]], [Tuesday, [...",(281) 807-4567,False,116241216200766427538,"[29.926546, -95.634197]"


In [7]:
df_data_places["price"] = df_data_places["price"].fillna("")
df_data_places = df_data_places[df_data_places['closed'] != True]
df_data_places = df_data_places.drop(columns = ["hours", "closed"])
df_data_places = df_data_places.dropna()

In [8]:
df_data_places

,name,price,address,phone,gPlusPlaceId,gps
0,Polo Club Cranbrook,,"[14531 Ella Blvd, Houston, TX 77014]",(281) 872-1222,111230142112807464303,"[29.979443, -95.439439]"
1,La Quinta Inn & Suites Austin Round Rock North,,"[2004 N Interstate Highway 35, Round Rock, TX ...",(512) 255-6666,111646207049589408629,"[30.52977, -97.691897]"
2,Coco Beach,,"[12159 Valliant St, San Antonio, TX 78216]",(210) 341-5330,100112847698256295619,"[29.552529, -98.49745]"
3,RadioShack,$$,"[4522 Fredericksburg Rd, San Antonio, TX 78201]",(800) 843-7422,106768567721430142078,"[29.492631, -98.552204]"
4,Hampton Inn & Suites Port Arthur,,"[7660 Memorial Blvd, Port Arthur, TX 77642]",(409) 722-6998,116191643129206359222,"[29.939667, -93.98328]"
...,...,...,...,...,...,...
115158,Cinco Ranch Veterinary Hospital,,"[2519 Cinco Park Pl, Katy, TX 77494]",(281) 693-7387,100738908015778376008,"[29.74484, -95.770493]"
115159,WINX Lash and Makeup Studio,,"[1973 W Gray St #23, Houston, TX 77019]",(832) 275-9667,108538528430034252187,"[29.752956, -95.406743]"
115160,Eastside Tattoo And Piercing Studio,,"[4819 Columbia Ave, Dallas, TX 75226]",(214) 828-9555,110544559460583713987,"[32.794887, -96.76422]"
115161,Fritz's Restaurant,,"[105 Ruthlynn Dr, Longview, TX 75605]",(903) 553-1161,110967995566557704067,"[32.530141, -94.743552]"


In [9]:
df_data_places = df_data_places.reset_index().drop(columns = ["index"])

In [10]:
df_data_places.to_csv("data_places.csv")

In [11]:
data_reviews = []
with gzip.open("data/reviews_TX.json.gz") as f:
    for l in f:
        data_reviews.append(eval(l))

In [12]:
data_reviews[:1], len(data_reviews)

([{'rating': 5.0,
   'reviewerName': 'Johanna S',
   'reviewText': "My husband uses his truck for work, and several months ago we started having problems with it. We needed it fixed ASAP in order for him to get back to work. It was a Thursday and I called the closest dealership, Lithia in Corpus, and they said they wouldn't be able to even look at it until the following Wednesday. So I called Port Lavaca Dodge. They told me to bring it that day, and they would do their best to get it looked at.  Long story short, the truck was up and running good as new by the following Wednesday. That's why we continue to drive the extra hour to Port Lavaca to have both our trucks serviced there.",
   'categories': ['Car Dealer'],
   'gPlusPlaceId': '104231480695579463827',
   'unixReviewTime': 1362944666,
   'reviewTime': 'Mar 10, 2013',
   'gPlusUserId': '100000050936341836015'}],
 542838)

In [13]:
df_data_reviews = pd.DataFrame.from_dict(data_reviews)

In [14]:
df_data_reviews

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId
0,5.0,Johanna S,"My husband uses his truck for work, and severa...",[Car Dealer],104231480695579463827,1.362945e+09,"Mar 10, 2013",100000050936341836015
1,5.0,Jason Wagner,"I don't live in Austin (I live in Cali), but I...","[Cigar Shop, Tobacco Shop, Gift Shop]",107589918909616947159,1.343283e+09,"Jul 25, 2012",100000196778399872657
2,5.0,Qui Williams,"Amazing Law Office kind hearted,supportive and...","[Personal Injury Attorney, Trial Attorney]",113483539975846506079,1.355097e+09,"Dec 9, 2012",100000409322409027563
3,5.0,Savanna Bowman,"WHEN I MET HANK AND GRAHAM OF SUTLIFF & STOUT,...","[Insurance Attorney, Personal Injury Attorney,...",113939686253574180930,1.342777e+09,"Jul 20, 2012",100000415460821501051
4,5.0,midengineZ06,"Great place to find tools, prices are.better.",None,118342244559062929091,NaN,None,100000420304795534622
...,...,...,...,...,...,...,...,...
542833,4.0,jeannette basquin,"Prices are decent, beds are great, very helpfu...",[Tanning Salon],103974695459972557518,1.337261e+09,"May 17, 2012",118446692227028504446
542834,4.0,jeannette basquin,This Starbucks is THEE BEST! Especially when i...,[Coffee Shop],104012221781323448380,1.339211e+09,"Jun 8, 2012",118446692227028504446
542835,5.0,jeannette basquin,It's really too bad this place is no longer op...,[Tanning Salon],107556450036182366224,1.337260e+09,"May 17, 2012",118446692227028504446
542836,4.0,jeannette basquin,If it's tanning you want It's tanning you get....,"[Day Spa, Tanning Salon]",115017223158347773445,1.337261e+09,"May 17, 2012",118446692227028504446


In [15]:
len(df_data_reviews)

542838

In [16]:
df_reviews = df_data_reviews.dropna()

In [17]:
joined_df = pd.merge(df_data_reviews, df_data_places, on='gPlusPlaceId', how='inner')

In [18]:
len(joined_df)

518710

In [19]:
joined_df

,rating,reviewerName,reviewText,categories,gPlusPlaceId,unixReviewTime,reviewTime,gPlusUserId,name,price,address,phone,gps
0,5.0,Johanna S,"My husband uses his truck for work, and severa...",[Car Dealer],104231480695579463827,1.362945e+09,"Mar 10, 2013",100000050936341836015,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]"
1,5.0,LaVon Garcia,"I purchased a Jeep Commander in 2010, it seems...",[Car Dealer],104231480695579463827,1.373040e+09,"Jul 5, 2013",100073644778479617329,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]"
2,5.0,Lindsay Hamilton,My dad purchased my first vehicle years ago fr...,[Car Dealer],104231480695579463827,1.363645e+09,"Mar 18, 2013",101122792075974600842,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]"
3,5.0,Larry Bandstra,This review is for the service department. Ser...,[Car Dealer],104231480695579463827,1.370173e+09,"Jun 2, 2013",102354832062937656279,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]"
4,1.0,javier campos,Beware!!! They advertise all these great deal...,[Car Dealer],104231480695579463827,1.378608e+09,"Sep 7, 2013",102440562308553176511,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
518705,4.0,Angie Gomez,Heritage Christian has been such a loving plac...,[Day Care Center],100874018747238623940,1.338583e+09,"Jun 1, 2012",118445242987118960023,Heritage Christian Day Care,,"[315 W House St, Alvin, TX 77511]",(281) 331-6082,"[29.418104, -95.246337]"
518706,5.0,Angie Gomez,My son came here when he was 6weeks old since ...,[Day Care Center],102305041483072012234,1.308086e+09,"Jun 14, 2011",118445242987118960023,Precious Stones Preschool and Daycare,,"[1407 Victory Ln, Alvin, TX 77511]",(281) 756-0990,"[29.452281, -95.229253]"
518707,5.0,Douglass Colzani,Sandy is awesome and always does a good job on...,[Beauty Salon],118290791164722075498,1.347683e+09,"Sep 14, 2012",118445437378196457095,Cutters Unlimited Styling,,"[136 Mary Lou Dr, Del Rio, TX 78840]",(830) 775-4092,"[29.390763, -100.907407]"
518708,5.0,James Gardner,Could not have been happier with my experience...,"[Guitar Store, Professional Services]",102279729296645274470,1.331085e+09,"Mar 6, 2012",118446077842935124479,Musical Palette,,"[6242 Rufe Snow Dr, Fort Worth, TX 76148]",(817) 428-2665,"[328.6037, -972.3801]"


In [20]:
joined_df["dtime"] = pd.to_datetime(joined_df["unixReviewTime"], unit="s")
joined_df = joined_df.drop(columns = ["reviewTime", "unixReviewTime", "reviewerName", "gPlusUserId"])

In [21]:
joined_df = joined_df.dropna()

In [22]:
joined_df = joined_df.reset_index().drop(columns = ["index"])

In [23]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 388427 entries, 0 to 388426
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   rating        388427 non-null  float64       
 1   reviewText    388427 non-null  object        
 2   categories    388427 non-null  object        
 3   gPlusPlaceId  388427 non-null  object        
 4   name          388427 non-null  object        
 5   price         388427 non-null  object        
 6   address       388427 non-null  object        
 7   phone         388427 non-null  object        
 8   gps           388427 non-null  object        
 9   dtime         388427 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(8)
memory usage: 29.6+ MB


In [24]:
joined_df.to_csv("data.csv", index=True)

### Text Analysis

In [25]:
import re
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences

In [26]:
TextBlob('Its a great place.').sentiment

Sentiment(polarity=0.8, subjectivity=0.75)

In [27]:
def MSE(predictions, labels):
    differences = [(x - y) ** 2 for x, y in zip(predictions, labels)]
    return sum(differences) / len(differences)
def MAE(predictions, labels):
    diff = [abs(x-y) for x, y in zip(predictions, labels)]
    return sum(diff)/ len(diff)

In [28]:
def preprocess_joined_data(df_in):
    df_in["reviewText"] = [item + "." for item in df_in["reviewText"]]
    df_in["reviewLength"] = [len(item) for item in df_in["reviewText"]]
    df_in["reviewHour"] = [item.hour for item in df_in["dtime"]]
    df_in["priceRank"] = [1 if price == '$' else 2 if price == '$$' else 3 if price == '$$$' else 0 for price in df_in['price']]

    df_in['sentences'] = df_in['reviewText'].apply(split_into_sentences)
    df_in['sentiments'] = df_in.apply(lambda row: [TextBlob(sentence).sentiment[0] for sentence in row['sentences']] if len(row['sentences']) > 0 else [TextBlob(row['reviewText']).sentiment[0]], axis=1)

    count_pos = df_in['sentiments'].apply(lambda sentiments: sum(sentiment > 0 for sentiment in sentiments))
    count_neg = len(df_in['sentiments'][0]) - count_pos

    df_in['Pos_prop'] = count_pos / len(df_in['sentiments'][0])
    df_in['Neg_prop'] = count_neg / len(df_in['sentiments'][0])

In [29]:
preprocess_joined_data(joined_df)

In [30]:
joined_df

,rating,reviewText,categories,gPlusPlaceId,name,price,address,phone,gps,dtime,reviewLength,reviewHour,priceRank,sentences,sentiments,Pos_prop,Neg_prop
0,5.0,"My husband uses his truck for work, and severa...",[Car Dealer],104231480695579463827,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]",2013-03-10 19:44:26,622,19,0,"[My husband uses his truck for work, and sever...","[0.0, 0.05, 0.25, 0.0, 1.0, 0.1572727272727272...",0.500,0.500
1,5.0,"I purchased a Jeep Commander in 2010, it seems...",[Car Dealer],104231480695579463827,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]",2013-07-05 15:57:20,900,15,0,"[I purchased a Jeep Commander in 2010, it seem...","[0.024999999999999994, 0.0, -0.1, 0.2, 0.0, 0....",0.500,0.500
2,5.0,My dad purchased my first vehicle years ago fr...,[Car Dealer],104231480695579463827,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]",2013-03-18 22:15:04,297,22,0,[My dad purchased my first vehicle years ago f...,"[0.25, 0.2571428571428571, 0.0]",0.250,0.750
3,5.0,This review is for the service department. Ser...,[Car Dealer],104231480695579463827,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]",2013-06-02 11:42:17,127,11,0,"[This review is for the service department., S...","[0.0, 0.625, 0.125, 0.0]",0.250,0.750
4,1.0,Beware!!! They advertise all these great deal...,[Car Dealer],104231480695579463827,Port Lavaca Dodge Chrysler Jeep Ram,,"[1901 Texas 35, Port Lavaca, TX 77979]",(800) 908-8495,"[28.612878, -96.645718]",2013-09-08 02:46:57,266,2,0,"[Beware!, !, !, They advertise all these great...","[0.0, 0.0, 0.0, 0.0500000000000001, 0.0, -0.03...",0.125,0.875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
388422,4.0,Heritage Christian has been such a loving plac...,[Day Care Center],100874018747238623940,Heritage Christian Day Care,,"[315 W House St, Alvin, TX 77511]",(281) 331-6082,"[29.418104, -95.246337]",2012-06-01 20:44:14,1704,20,0,[Heritage Christian has been such a loving pla...,"[0.0875, -0.30476190476190473, 0.0, 0.4875, -0...",1.500,-0.500
388423,5.0,My son came here when he was 6weeks old since ...,[Day Care Center],102305041483072012234,Precious Stones Preschool and Daycare,,"[1407 Victory Ln, Alvin, TX 77511]",(281) 756-0990,"[29.452281, -95.229253]",2011-06-14 21:15:36,824,21,0,[My son came here when he was 6weeks old since...,"[0.05, 0.0, 0.6, 0.0, 0.2, 0.5, 0.375, 0.43333...",1.125,-0.125
388424,5.0,Sandy is awesome and always does a good job on...,[Beauty Salon],118290791164722075498,Cutters Unlimited Styling,,"[136 Mary Lou Dr, Del Rio, TX 78840]",(830) 775-4092,"[29.390763, -100.907407]",2012-09-15 04:17:42,58,4,0,[Sandy is awesome and always does a good job o...,[0.7333333333333334],0.125,0.875
388425,5.0,Could not have been happier with my experience...,"[Guitar Store, Professional Services]",102279729296645274470,Musical Palette,,"[6242 Rufe Snow Dr, Fort Worth, TX 76148]",(817) 428-2665,"[328.6037, -972.3801]",2012-03-07 01:45:23,447,1,0,[Could not have been happier with my experienc...,"[0.0, -0.4333333333333334, 0.0, 0.150000000000...",0.250,0.750


In [31]:
joined_df.to_csv("splited.csv")

In [32]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(joined_df, joined_df["rating"], test_size=0.1, random_state=42)

### Having two features - proportions of positive and negative sentence types in review text -> rating - This is the first model in model-03.ipynb
Ignoring the reviews which can't be split into sentences.

### Having two features - proportions of positive and negative sentence types in review text - Extension 1
Modification - Considering all the reviews which can't be split into sentences

In [33]:
selected_columns = ["Pos_prop", "Neg_prop"]
X = X_train[selected_columns].copy()
y = y_train
X_te = X_test[selected_columns].copy()
y_test

53767     5.0
94429     4.0
80122     5.0
164090    5.0
105250    1.0
         ... 
80298     2.0
370894    1.0
52669     2.0
116541    3.0
196972    1.0
Name: rating, Length: 38843, dtype: float64

In [34]:
len(X), len(y)

(349584, 349584)

In [35]:
X

,Pos_prop,Neg_prop
172163,0.500,0.500
113414,0.250,0.750
92628,0.250,0.750
244149,0.000,1.000
191274,0.125,0.875
...,...,...
259178,0.000,1.000
365838,0.625,0.375
131932,0.250,0.750
146867,0.375,0.625


In [36]:
clf = linear_model.Ridge(1.0, fit_intercept=False)  # MSE + 1.0 l2
clf.fit(X, y)
theta = clf.coef_
pred = clf.predict(X)
pred_test = clf.predict(X_te)

In [37]:
theta

array([4.57754505, 3.70959121])

In [38]:
pred[:5]

array([4.14356813, 3.92657967, 3.92657967, 3.70959121, 3.81808544])

In [39]:
any(value > 5  for value in pred)

True

In [40]:
MSE(pred, y), MSE(pred_test, y_test)

(1.8370852569857483, 1.8511439622045176)

In [41]:
MAE(pred, y), MAE(pred_test, y_test)

(1.124308809319306, 1.128885061221842)

### Having two features - proportions of positive and negative sentence types in review text along with hour, length, price (encoded) - Extension 2 

In [42]:
selected_columns = ["Pos_prop", "Neg_prop", "reviewLength", "priceRank", "reviewHour"]
X = X_train[selected_columns].copy()
y = y_train
X_te = X_test[selected_columns].copy()
y_test

53767     5.0
94429     4.0
80122     5.0
164090    5.0
105250    1.0
         ... 
80298     2.0
370894    1.0
52669     2.0
116541    3.0
196972    1.0
Name: rating, Length: 38843, dtype: float64

In [43]:
X[:5]

,Pos_prop,Neg_prop,reviewLength,priceRank,reviewHour
172163,0.500,0.500,381,0,0
113414,0.250,0.750,495,2,13
92628,0.250,0.750,134,2,1
244149,0.000,1.000,41,0,2
191274,0.125,0.875,110,0,2


In [44]:
clf = linear_model.Ridge(1.0, fit_intercept=False)  # MSE + 1.0 l2
clf.fit(X, y)
theta = clf.coef_
pred = clf.predict(X)
pred_test = clf.predict(X_te)

In [45]:
theta

array([ 7.23177299e+00,  3.84992572e+00, -2.52295756e-03, -1.52589608e-01,
        2.92683883e-03])

In [46]:
pred[:5]

array([4.57960252, 3.17939323, 4.05505885, 3.75233813, 4.00098497])

In [47]:
any(value > 5  for value in pred)

True

In [48]:
MSE(pred, y), MSE(pred_test, y_test)

(1.365136849764089, 1.3729513206343653)

In [49]:
MAE(pred, y), MAE(pred_test, y_test)

(0.9397599604869741, 0.9430437927460759)

In [50]:
import joblib
joblib.dump(clf, './models/model.joblib')

['./models/model.joblib']

### Stat

In [51]:
import pandas as pd
from nltk import pos_tag, word_tokenize
from nltk.corpus import stopwords
from collections import Counter

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [52]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import stopwords
from collections import Counter

def top_nouns_by_sentiment_range(df, text_column, sentiment_column, id_column, negative_range, positive_range, n=20):
    noun_freq_by_id_sentiment_range = {}

    for index, row in df.iterrows():
        sentiments = row[sentiment_column]
        if not sentiments:
            continue

        text_data = row[text_column]
        if not text_data:
            continue

        place_id = row[id_column]
        place_name = row["name"]

        if place_id not in noun_freq_by_id_sentiment_range:
            noun_freq_by_id_sentiment_range[place_id] = {
                'negative': Counter(),
                'positive': Counter()
            }

        for i, sentiment in enumerate(sentiments):
            if negative_range[0] <= sentiment < negative_range[1]:
                sentiment_label = 'negative'
            elif positive_range[0] <= sentiment <= positive_range[1]:
                sentiment_label = 'positive'
            else:
                continue

            tokens = word_tokenize(text_data[i])
            tagged_words = pos_tag(tokens)

            nouns = [word for word, pos in tagged_words if pos.startswith('N')]
            stop_words = set(stopwords.words('english'))
            nouns = [word.lower() for word in nouns if word.lower() not in stop_words]

            noun_freq_by_id_sentiment_range[place_id][sentiment_label].update(nouns)

    # Extract the top n items for each sentiment category
    top_nouns_by_id_sentiment_range = {}
    for place_id, sentiment_counts in noun_freq_by_id_sentiment_range.items():
        top_nouns_by_id_sentiment_range[place_id] = {}
        for sentiment_label, noun_counter in sentiment_counts.items():
            top_nouns_by_id_sentiment_range[place_id][sentiment_label] = dict(noun_counter.most_common(n))
        top_nouns_by_id_sentiment_range[place_id]['name'] = place_name
        

    return top_nouns_by_id_sentiment_range

# Specify the columns for sentiment analysis, sentiment values range, and ID column
text_column_for_analysis = 'sentences'
sentiment_column_for_analysis = 'sentiments'
id_column_for_analysis = 'gPlusPlaceId'

# Define sentiment ranges
negative_range = (-1, 0)
positive_range = (0, 1)

# Get top 20 nouns for each sentiment range, grouped by ID
top_nouns_by_id_sentiment_range = top_nouns_by_sentiment_range(
    joined_df, text_column_for_analysis, sentiment_column_for_analysis, id_column_for_analysis, negative_range, positive_range, n=20
)

In [53]:
# Print top 20 nouns for each ID and sentiment
for place_id, sentiment_nouns in top_nouns_by_id_sentiment_range.items():
    print(f"Place ID: {place_id}")
    for sentiment_label, top_nouns in sentiment_nouns.items():
        print(f"  {sentiment_label.capitalize()} Top Nouns: {top_nouns}")
    print("\n")

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)




  Name Top Nouns: A Miracle to Be Hold LLC


Place ID: 102651441637149062601
  Negative Top Nouns: {'hotel': 2, 'staff': 2, 'base': 2, 'pizzeria': 2, 'rooms': 1, 'mildew': 1, 'toilet': 1, 'lid': 1, 'chain': 1, 'really': 1, 'makes': 1, 'angry': 1, 'acted': 1, 'big': 1, 'deal': 1, 'according': 1, 'leave': 1, 'something': 1, 'room': 1, 'responsible': 1}
  Positive Top Nouns: {'hotel': 6, 'room': 5, 'problems': 3, 'day': 2, 'night': 2, 'staff': 2, 'space': 2, 'days': 1, 'inn': 1, 'service': 1, 'reservation': 1, 'things': 1, 'deal': 1, 'last': 1, 'trip': 1, 'go': 1, 'get': 1, 'lunch': 1, 'mom': 1, 'accidently': 1}
  Name Top Nouns: A Miracle to Be Hold LLC


Place ID: 107004315812000907337
  Negative Top Nouns: {'southern': 1, 'maid': 1, 'locations': 1}
  Positive Top Nouns: {'place': 1, 'location': 1, 'neighborhood': 1, 'way': 1, 'kolaches': 1, 'city': 1}
  Name Top Nouns: A Miracle to Be Hold LLC


Place ID: 112392622051641049654
  Negative Top Nouns: {'furniture': 1, 'past': 1, 'problem

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [54]:
top_nouns_by_id_sentiment_range

{'104231480695579463827': {'negative': {'port': 2,
   'lavaca': 2,
   'experience': 2,
   'price': 2,
   'credit': 2,
   'visits': 1,
   'kristen': 1,
   'mueller': 1,
   'cab': 1,
   'trucks': 1,
   'hour': 1,
   'journey': 1,
   'something': 1,
   'things': 1,
   'rid': 1,
   'memory': 1,
   'sales': 1,
   'staff': 1,
   'concern': 1,
   'vehicle': 1},
  'positive': {'dodge': 14,
   'lavaca': 13,
   'service': 13,
   'port': 12,
   'truck': 6,
   'department': 6,
   'car': 6,
   'customer': 4,
   'beware': 4,
   'work': 3,
   'durango': 3,
   'kristen': 3,
   'mueller': 3,
   'carlos': 3,
   'rudy': 3,
   'oil': 3,
   'years': 3,
   'reason': 3,
   'approval': 3,
   'sales': 3},
  'name': 'A Miracle to Be Hold LLC'},
 '107589918909616947159': {'negative': {'cigars': 4,
   'bobalu': 4,
   'cigar': 3,
   'time': 2,
   'variety': 1,
   'sizes': 1,
   'flavors': 1,
   'go': 1,
   'sticks': 1,
   'sake': 1,
   'hell': 1,
   'smoke': 1,
   'shop': 1,
   'co': 1,
   'thanks': 1,
   'supplie

In [55]:
import pickle
with open('./data/counter.pkl', 'wb') as file:
    pickle.dump(top_nouns_by_id_sentiment_range, file)